In [34]:
import numpy as np
import pandas as pd
import random

In [55]:
def ScaleFreeMapGen(num_node):
    connection_map = np.zeros((num_node, num_node))
    scale_para = 1.8
    prob_sum = 0
    for i in range(1, num_node+1):
        prob_sum += 1/(i ** scale_para)
    # for Scale Free, the probability is the inverse of # of connection
    for i in range(num_node):
        # Find out how many out going connection of a given node
        rn = random.uniform(0, prob_sum)
        num_conn = 1
        while rn > 0:
            rn = rn - 1/(num_conn ** scale_para)
            num_conn += 1
            if num_conn == num_node:
                break
        # Which nodes the current nodes are connected to
        node_conn = random.sample(range(num_node), num_conn)
        # Put the weight into the connection matrix
        for j in node_conn:
            # weight from distribution uniform(-0.01, 0.01), can change here later to determine distribution
            weight = random.uniform(-0.01, 0.01)
            connection_map[i, j-1] = weight
    return connection_map
            
        
            

In [67]:

output_sf = ScaleFreeMapGen(10000)
output_sf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [73]:
def SmallWorldMapGen(num_node):
    connection_map = np.zeros((num_node, num_node))
    # Parameters for SmallWorld Network, can be changed later
    N = num_node
    m = int(np.log(num_node)) + 1 # number of neighbour
    M = N * m
    beta = 0.2
    # Find out what are some connections needs to be rewired
    num_rewireing = int(M * beta)
    conn_rewire = random.sample(range(M), num_rewireing)
    # Start connection, 
    counter = 0
    for i in range(num_node):
        for j in range(m):
            # weight from distribution uniform(-0.01, 0.01), can change here later to determine distribution
            weight = random.uniform(-0.01, 0.01)
            if counter in conn_rewire:
                #wire to a random one
                connection_map[i, (i + random.sample(range(1, num_node), 1)[0]) % num_node] = weight
            else:
                connection_map[i, (i + j) % num_node] = weight
            counter += 1
    return connection_map
            
    

In [76]:

output_sm = SmallWorldMapGen(10)
output_sm

array([[ 0.00578687,  0.00914507,  0.00486426,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.00759237,  0.001543  ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.00187088,  0.        ],
       [-0.00710546,  0.        ,  0.00149326, -0.00637903,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.00207133, -0.00150831,
         0.00680036,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.00674568,  0.00692445,
         0.00188342,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.0026354 , -0.00488678, -0.00548993,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.00578332,  0.00767828

In [106]:
def replaceNodewithNetwork(org_network, n, added_network):
    out_by_n = org_network[n, :]
    incoming_to_n = org_network[:, n]
    dim = org_network.shape[0] + added_network.shape[0]-1
    new_network = np.zeros((dim, dim))
    # Relocate
    added_dim = added_network.shape[0]
    new_network[:n,:n] = org_network[:n,:n]
    new_network[added_dim+n:, added_dim+n:] = org_network[n+1:, n+1:]
    new_network[:n, added_dim+n:] = org_network[:n, n+1:]
    new_network[added_dim+n: , :n] = org_network[n+1: , :n]
    # Put the added_network in
    new_network[n:added_dim+n, n:added_dim+n] = added_network
    # Randomly put incoming and outgoing connection into the added network
    idx = 0
    for i in out_by_n:
        if idx < n:
            rn = random.sample(range(n, n + added_dim), 1)[0]
            new_network[rn , idx] = i
        elif idx >n:
            rn = random.sample(range(n, n + added_dim), 1)[0]
            new_network[rn , idx + added_dim - 1] = i
        idx +=1
    idx = 0
    for i in incoming_to_n:
        if idx < n:
            rn = random.sample(range(n, n + added_dim), 1)[0]
            new_network[idx , rn] = i
        elif idx > n:
            rn = random.sample(range(n, n + added_dim), 1)[0]
            new_network[idx + added_dim - 1, rn] = i
        idx +=1
        
    return new_network

In [107]:
ones = np.ones((7,7))
zeros = np.zeros((3,3))
replaceNodewithNetwork(ones, 2, zeros)


array([[1., 1., 0., 1., 0., 1., 1., 1., 1.],
       [1., 1., 0., 0., 1., 1., 1., 1., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0.],
       [1., 1., 0., 0., 1., 1., 1., 1., 1.],
       [1., 1., 0., 0., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1.]])

In [97]:
zeros

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])